In [ ]:
{"id": 7306260, "ref": "aryan208/student-habits-and-academic-performance-dataset", "subtitle": "Analyze how habits and mental health shape student success.", "creatorName": "Aryan Kumar", "creatorUrl": "aryan208", "totalBytes": 2994172, "url": "https://www.kaggle.com/datasets/aryan208/student-habits-and-academic-performance-dataset", "lastUpdated": "2025-05-02T00:27:34.627Z", "downloadCount": 6614, "isPrivate": false, "isFeatured": false, "licenseName": "CC0: Public Domain", "description": "", "ownerName": "Aryan Kumar", "ownerRef": "aryan208", "kernelCount": 5, "title": "Student Habits and Academic Performance Dataset", "topicCount": 0, "viewCount": 26831, "voteCount": 58, "currentVersionNumber": 1, "usabilityRating": 1.0, "tags": [{"ref": "education", "name": "education", "description": "Education datasets and kernels relate to statistics on student performance, university rankings, and answers for all your homework questions.", "fullPath": "subject > people and society > education", "competitionCount": 60, "datasetCount": 35778, "scriptCount": 15142, "totalCount": 50980}, {"ref": "computer science", "name": "computer science", "description": "There is one thing we can say for sure about computer science: Computer science leads to more computer games which leads to better GPUs which leads to smarter AI.", "fullPath": "subject > science and technology > computer science", "competitionCount": 16, "datasetCount": 57897, "scriptCount": 776, "totalCount": 58689}], "files": [], "versions": []}

# Kaggle datasets

In [1]:
#Move the kaggle.json API file to the .kaggle directory for authentication
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

#Changing permissions on the file
!chmod 600 ~/.kaggle/kaggle.json

#Installing openml and xlswriter packages
#PD: xlswriter will be useful to treat links in the excel files
!pip install openml XlsxWriter --quiet

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.5 MB/s eta 0:00:0000:01


In [2]:
!pip freeze > requirements.txt

In [1]:
import pandas as pd
import openml, xlsxwriter, os

from warnings import filterwarnings

filterwarnings("ignore")
# kaggle.api.authenticate()

Creating the dataframe that will hold the name of the dataset, file name, description and link

In [7]:
main_records = pd.DataFrame(columns=['title', 'file-name', 'description', 'link'])
main_records.to_excel('datasets_information.xlsx',
                      index=None,
                      engine='xlsxwriter')

In [8]:
main_records = pd.read_excel('datasets_information.xlsx')
main_records

,title,file-name,description,link


Searching the datasets through the kaggle database

In [9]:
datasets_names = []

#keywords to search for datasets
search_terms = ['physics']

#Retrieving dataset names to be dowloaded
for word in search_terms:
    datasets = [kaggle.api.dataset_list(page=page, search=word, max_size=1_000) for page in range(1, 2)]
    for list_object in datasets:
        for datasetname in list_object:
            datasets_names.append(datasetname)

print(len(datasets_names))

11


In [ ]:
folder_path = "data"

for data in datasets_names:
    if (data.title not in list(main_records['title'])) \
        and (data.url not in list(main_records['link'])):

        #Licences of the datasets that will be allowed to be downloaded
        #In this case: public domain licences
        if vars(data)['licenseName'] in ["Database: Open Database", "Contents: Database Contents", \
                                        "CC0: Public Domain", "GLP", "GLP2", \
                                        "Community Data License Agreement - Sharing - Version 1.0", \
                                        "GNU Lesser General Public License 3.0"]:

            #Information that we will want to be in the final dataframe
            title = vars(data)['title']
            file_name = str(data).split('/')[1]
            description = kaggle.api.dataset_view(data.ref).description
            link = vars(data)['url']

            main_records = pd.concat([main_records,
                                      pd.DataFrame({'title': [title],
                                                    'file-name': [file_name],
                                                    'description': [description],
                                                    'link':[link]})])

            #Downloading the data
            directory = '/content/data'
            if not os.path.exists(directory):
                os.makedirs(directory)
            
            kaggle.api.dataset_download_files(data.ref,
                                              path='/content/data',
                                              unzip=False)

            archivo_zip = str(data).split('/')[1] + '.zip'
            folder_name = os.path.splitext(archivo_zip)[0]

        else: pass
    else: pass

main_records.to_excel('datasets_information.xlsx',
                      index=None,
                      engine='xlsxwriter')

Backup code to save the dataframe to excel file if the above code throws a warning or error

In [11]:
main_records.to_excel('datasets_information.xlsx',
                      index=None,
                      engine='xlsxwriter')

# OpenML datasets

Mainly, this OpenML database contain supervised learning datasets

Creation of the dataframe that will contain the files information

In [2]:
openml_records = pd.DataFrame(columns=['title', 'file-name', 'description', 'link'])
openml_records.to_excel('./openml_datasets_information.xlsx',
                        index=None,
                        engine="xlsxwriter")

In [3]:
openml_records = pd.read_excel('openml_datasets_information.xlsx')
openml_records

,title,file-name,description,link


In [4]:
range_of_index = range(30, 45)

for dataset_index in range_of_index:
    try:
        dataset_info = openml.datasets.get_dataset(dataset_id=dataset_index, download_data=False)
        if (dataset_info.name not in list(openml_records['title'])) \
        and (dataset_info.url not in list(openml_records['link'])):

            title = dataset_info.name
            description = dataset_info.description
            link = dataset_info.original_data_url

            openml_records = pd.concat([openml_records,
                                        pd.DataFrame({'title': [title],
                                                      'file-name': [title],
                                                      'description': [description],
                                                      'link':[link]})])

            #Turning the data into a dataframe format
            X, y, categorical_indicator, attribute_names = dataset_info.get_data(
            dataset_format="array", target=dataset_info.default_target_attribute)
            df = pd.DataFrame(X, columns=attribute_names)
            df["class"] = y

            directory = '/content/openmldata'
            if not os.path.exists(directory):
                os.makedirs(directory)

            df.to_csv('/content/openmldata/' + title + '.csv')
        else: pass

    except openml.exceptions.OpenMLPrivateDatasetError:
        print(f"The dataset {dataset_index} is private and cannot be accessed.")
        pass
openml_records.to_excel('openml_datasets_information.xlsx',
                        index=None,
                        engine='xlsxwriter')

Could not download file from http://openml1.win.tue.nl/dataset33/dataset_33.pq: HTTPConnectionPool(host='openml1.win.tue.nl', port=80): Max retries exceeded with url: /dataset33/dataset_33.pq (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


In [15]:
openml_records.to_excel('openml_datasets_information.xlsx',
                        index=None,
                        engine='xlsxwriter')

In [1]:
!pipreqs --force

INFO: Successfully saved requirements file in d:\Estiven\Datos\Proyectos\kaggle_openml_api\requirements.txt
